# AutoGluon Tabular with SageMaker

[AutoGluon](https://github.com/awslabs/autogluon) automates machine learning tasks enabling you to easily achieve strong predictive performance in your applications. With just a few lines of code, you can train and deploy high-accuracy deep learning models on tabular, image, and text data.
This notebook shows how to use AutoGluon-Tabular with Amazon SageMaker by creating custom containers.

## Prerequisites

If using a SageMaker hosted notebook, select kernel `conda_mxnet_p36`.

In [2]:
# Make sure docker compose is set up properly for local mode
!./setup.sh

The user has root access.
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


In [3]:
# Imports
import os
import boto3
import sagemaker
from time import sleep
from collections import Counter
import pandas as pd
from sagemaker import get_execution_role, local, Model
from sagemaker.estimator import Estimator
from sagemaker.predictor import RealTimePredictor, csv_serializer, StringDeserializer
from sklearn.metrics import accuracy_score, classification_report
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell

# Print settings
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10)

# Account/s3 setup
session = sagemaker.Session()
local_session = local.LocalSession()
bucket = session.default_bucket()
prefix = 'sagemaker/autogluon-tabular'
region = session.boto_region_name
role = get_execution_role()
client = boto3.client('sts')
account = client.get_caller_identity()['Account']

### Build docker images

First, build autogluon package to copy into docker image.

In [4]:
if not os.path.exists('package'):
    !pip install PrettyTable -t package
    !pip install bokeh -t package
    !pip install --pre autogluon==0.0.6 -t package
    !pip install numpy==1.16.1 -t package    
    !pip install --upgrade boto3 -t package
    !pip install bokeh -t package
    !pip install --upgrade matplotlib -t package

Now build the training/inference image and push to ECR

In [5]:
training_algorithm_name = 'autogluon-sagemaker-training'
inference_algorithm_name = 'autogluon-sagemaker-inference'

In [6]:
!./container-training/build_push_training.sh {training_algorithm_name}
!./container-inference/build_push_inference.sh {inference_algorithm_name}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon    701MB
Step 1/9 : ARG REGION=us-east-1
Step 2/9 : FROM 763104351884.dkr.ecr.${REGION}.amazonaws.com/mxnet-training:1.6.0-cpu-py3
 ---> 6eb60fab1b5a
Step 3/9 : RUN pip install --upgrade pip
 ---> Using cache
 ---> 6d38331d1216
Step 4/9 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 55363446fae1
Step 5/9 : COPY package

### Get the data

In this example we'll use the direct-marketing dataset to build a binary classification model that predicts whether customers will accept or decline a marketing offer.  
First we'll download the data and split it into train and test sets. AutoGluon does not require a separate validation set (it uses bagged k-fold cross-validation).

In [7]:
# Download and unzip the data
!wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip --quiet
!unzip -qq -o bank-additional.zip
!rm bank-additional.zip

local_data_path = './bank-additional/bank-additional-full.csv'
data = pd.read_csv(local_data_path)

# Split train/test data
train = data.sample(frac=0.7, random_state=42)
test = data.drop(train.index)

# Split test X/y
label = 'y'
y_test = test[label]
X_test = test.drop(columns=[label])

##### Check the data

In [8]:
train.head(3)
train.shape

test.head(3)
test.shape

X_test.head(3)
X_test.shape

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
32884,57,technician,married,high.school,no,no,yes,cellular,may,mon,371,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
3169,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,285,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
32206,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,52,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no


(28832, 21)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
9,25,services,single,high.school,no,yes,no,telephone,may,mon,50,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
10,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,55,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


(12356, 21)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
9,25,services,single,high.school,no,yes,no,telephone,may,mon,50,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
10,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,55,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0


(12356, 20)

Upload the data to s3

In [9]:
train_file = 'train.csv'
train.to_csv(train_file,index=False)
train_s3_path = session.upload_data(train_file, key_prefix='{}/data'.format(prefix))

test_file = 'test.csv'
test.to_csv(test_file,index=False)
test_s3_path = session.upload_data(test_file, key_prefix='{}/data'.format(prefix))

X_test_file = 'X_test.csv'
X_test.to_csv(X_test_file,index=False)
X_test_s3_path = session.upload_data(X_test_file, key_prefix='{}/data'.format(prefix))

## Train

The minimum requirement for hyperparameters is a target label.

In [10]:
hyperparameters = {'label': 'y'}

##### (Optional) hyperparameters can be passed to the `autogluon.task.TabularPrediction.fit` function.  

Below shows AutoGluon hyperparameters from the example [Predicting Columns in a Table - In Depth](https://autogluon.mxnet.io/tutorials/tabular_prediction/tabular-indepth.html#model-ensembling-with-stacking-bagging). Please see [fit parameters](https://autogluon.mxnet.io/api/autogluon.task.html?highlight=eval_metric#autogluon.task.TabularPrediction.fit) for further information.


Here's a more in depth example from the above tutorial that shows how to provide hyperparameter ranges and additional settings:

```python
nn_options = {
    'num_epochs': '10',
    'learning_rate': "ag.space.Real(1e-4, 1e-2, default=5e-4, log=True)",
    'activation': "ag.space.Categorical('relu', 'softrelu', 'tanh')",
    'layers': "ag.space.Categorical([100],[1000],[200,100],[300,200,100])",
    'dropout_prob': "ag.space.Real(0.0, 0.5, default=0.1)"
}

gbm_options = {
    'num_boost_round': '100',
    'num_leaves': "ag.space.Int(lower=26, upper=66, default=36)"
}

model_hps = {'NN': nn_options, 'GBM': gbm_options} 

hyperparameters = {
    'label': 'y',
    'time_limits': 2*60,
    'hyperparameters': model_hps,
    'auto_stack': False,    
    'hyperparameter_tune': True,
    'search_strategy': 'skopt'
}
```
**Note:** Your hyperparameter choices may affect the size of the model package, which could result in additional time taken to upload your model and complete training.

<br>

For local training set `train_instance_type` to `local` .  
For non-local training the recommended instance type is `ml.m5.2xlarge` .

In [11]:
%%time

instance_type = 'ml.m5.2xlarge'
#instance_type = 'local'

ecr_image = f'{account}.dkr.ecr.{region}.amazonaws.com/{training_algorithm_name}:latest'

estimator = Estimator(image_name=ecr_image,
                      role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      hyperparameters=hyperparameters)

estimator.fit(train_s3_path)

2020-04-08 03:57:43 Starting - Starting the training job...
2020-04-08 03:57:44 Starting - Launching requested ML instances...
2020-04-08 03:58:39 Starting - Preparing the instances for training......
2020-04-08 03:59:31 Downloading - Downloading input data
2020-04-08 03:59:31 Training - Downloading the training image......
2020-04-08 04:00:34 Training - Training image download completed. Training in progress.2020-04-08 04:00:34,785 sagemaker-containers INFO     Imported framework sagemaker_mxnet_container.training
2020-04-08 04:00:34,786 sagemaker-containers INFO     Failed to parse hyperparameter label value y to Json.
Returning the value itself
2020-04-08 04:00:34,788 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-08 04:00:34,802 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"label":"y"}', 'SM_USER_ENTRY_POINT': 'train.py', 'SM_FRAMEWORK_PARAMS'

### Create Model

In [12]:
# Create predictor object
class AutoGluonTabularPredictor(RealTimePredictor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, content_type='text/csv', 
                         serializer=csv_serializer, 
                         deserializer=StringDeserializer(), **kwargs)

In [13]:
ecr_image = f'{account}.dkr.ecr.{region}.amazonaws.com/{inference_algorithm_name}:latest'

if instance_type == 'local':
    model = estimator.create_model(image=ecr_image, role=role)
else:
    model_uri = os.path.join(estimator.output_path, estimator._current_job_name, "output", "model.tar.gz")
    model = Model(model_uri, ecr_image, role=role, sagemaker_session=session, predictor_cls=AutoGluonTabularPredictor)

### Batch Transform

For local mode, either `s3://<bucket>/<prefix>/output/` or `file:///<absolute_local_path>` can be used as outputs.

By including the label column in the test data, you can also evaluate prediction performance (In this case, passing `test_s3_path` instead of `X_test_s3_path`).

In [14]:
output_path = f's3://{bucket}/{prefix}/output/'
# output_path = f'file://{os.getcwd()}'

transformer = model.transformer(instance_count=1, 
                                instance_type=instance_type,
                                strategy='SingleRecord',
                                max_payload=100,
                                max_concurrent_transforms=1,                              
                                output_path=output_path)

transformer.transform(test_s3_path, content_type='text/csv')
transformer.wait()

..................
2020-04-08 04:06:39,356 [INFO ] main com.amazonaws.ml.mms.ModelServer - 
MMS Home: /usr/local/lib/python3.6/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 8
Max heap size: 6295 M
Python executable: /usr/local/bin/python3.6
Config file: /etc/sagemaker-mms.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Model Store: /.sagemaker/mms/models
Initial Models: ALL
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 8
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
2020-04-08 04:06:39,399 [INFO ] main com.amazonaws.ml.mms.wlm.ModelManager - Model model loaded.
2020-04-08 04:06:39,414 [INFO ] main com.amazonaws.ml.mms.ModelServer - Initialize Inference server with: EpollServerSocketChannel.
2020-04-08 04:06:39,637 [INFO ] W-9005-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Listening

In [15]:
# Check s3 for <filename>.csv.out file
if instance_type != 'local':
    !aws s3 ls {transformer.output_path} --recursive
elif 's3' in output_path:
    !aws s3 ls {output_path+transformer.latest_transform_job.job_name} --recursive

2020-03-22 16:01:23   28303280 sagemaker/autogluon-tabular/output/AlgorithmValidation-1fb5ca16-1f58-4114-8d91-de4022244914-1/output/model.tar.gz
2020-03-22 16:11:08   59329494 sagemaker/autogluon-tabular/output/AlgorithmValidation-282ccc17-0718-4cdc-8b91-3dbb0b5de269-1/output/model.tar.gz
2020-03-22 16:43:50   59345824 sagemaker/autogluon-tabular/output/AlgorithmValidation-8ef7d77b-f1e9-4c42-8828-7380f713a8a8-1/output/model.tar.gz
2020-03-22 15:51:53   16887662 sagemaker/autogluon-tabular/output/AlgorithmValidation-b83b0cc1-2a04-4238-be24-da2725747229-1/output/model.tar.gz
2020-03-22 15:37:26   13670151 sagemaker/autogluon-tabular/output/AlgorithmValidation-ea0955ba-b3d2-4f30-9dec-9108674ac407-1/output/model.tar.gz
2020-03-25 06:59:18      12744 sagemaker/autogluon-tabular/output/X_test.csv.out
2020-03-19 23:24:55      25123 sagemaker/autogluon-tabular/output/autogluon-sagemaker-inference-2020-03-1-2020-03-19-23-24-37-181/test.csv.out
2020-03-24 07:24:19         33 sagemaker/autogluon-

### Endpoint

##### Deploy remote or local endpoint

In [16]:
instance_type = 'ml.m5.2xlarge'
#instance_type = 'local'

predictor = model.deploy(initial_instance_count=1, 
                         instance_type=instance_type)

Using already existing model: autogluon-sagemaker-inference-2020-04-08-04-03-56-043


-------------!

##### Attach to endpoint (or reattach if kernel was restarted)

In [17]:
# Select standard or local session based on instance_type
if instance_type == 'local': sess = local_session
else: sess = session

# Attach to endpoint
predictor = AutoGluonTabularPredictor(predictor.endpoint, sagemaker_session=sess)

##### Predict on unlabeled test data

In [18]:
results = predictor.predict(X_test.to_csv())

# Check output
print(Counter(results.splitlines()))

Counter({'no': 11285, 'yes': 1071})


##### Predict on data that includes label column  
Prediction performance metrics will be printed to endpoint logs.

In [19]:
results = predictor.predict(test.to_csv())

# Check output
sleep(0.1); print(Counter(results.splitlines()))

Counter({'no': 11285, 'yes': 1071})


##### Check that performance metrics match evaluation printed to endpoint logs as expected

In [20]:
import numpy as np
y_results = np.array(results.splitlines())

print("accuracy: {}".format(accuracy_score(y_true=y_test, y_pred=y_results)))
print(classification_report(y_true=y_test, y_pred=y_results, digits=6))

accuracy: 0.9191485917772741
              precision    recall  f1-score   support

          no   0.941338  0.969252  0.955091     10960
         yes   0.685341  0.525788  0.595055      1396

   micro avg   0.919149  0.919149  0.919149     12356
   macro avg   0.813339  0.747520  0.775073     12356
weighted avg   0.912415  0.919149  0.914414     12356



##### Clean up endpoint

In [21]:
predictor.delete_endpoint()